Testing the sample size needed to get an accurate/ stable standard deviation value

In [19]:
import matplotlib.pyplot as plt
import numpy as np
# imports my own created library
import ChargedDisk.Simulation as sim
from ChargedDisk.BaseFuncs import Mult_pass as MP
from ChargedDisk.BaseFuncs import Single_pass as SP

# random walk sim
def walk(walkers: int, steps = 100, dims = 3, delta = 1):
    walks = 2*np.random.rand(walkers,steps, dims)-1
    walks *= delta
    dist = np.cumsum(walks, axis = 1)
    return dist


''' This whole piece of code was used to try and increase the accuracy of dervied values but ran into major Ram issues as discussed'''
disk_radius = 100
num_charges = 20
accuracies = np.arange(6,7, 1)
reruns = 10
sigma_lst = np.zeros((reruns,accuracies.shape[0]))
for i in range(reruns):
    for index, accuracy in enumerate(accuracies):
        sample_size = np.power(10,accuracy)
        charges = MP.gen_coords(sample_size, disk_radius, num_charges)
        if accuracy >= 6:
            #splitting the array to reduce the size of arrays
            split_arr = np.split(charges,int(sample_size/np.power(10,6)))
            energies = np.squeeze(np.round(MP.calc_energy(split_arr[0], charge=1), accuracy))
            for arr in range(1,len(split_arr)):        
                energies = np.concatenate((energies,np.squeeze(np.round(MP.calc_energy(split_arr[arr], charge=1), accuracy)))) # calculate the energies to the degree of accuracy
        else:
            energies = np.squeeze(np.round(MP.calc_energy(charges, charge=1), accuracy))
        
        sigma = np.std(energies) # calculate the standard deviation
        sigma_lst[i,index] = sigma
    print('complete run {}'.format(i+1))
print(sigma_lst)
print(np.std(sigma_lst[:,-1]))

complete run 1
complete run 2
complete run 3
complete run 4
complete run 5
complete run 6
complete run 7
complete run 8
complete run 9
complete run 10
[[1.50487351]
 [1.42937836]
 [1.49671293]
 [1.56377803]
 [1.81836331]
 [1.46335913]
 [1.43539889]
 [1.56309545]
 [1.50952878]
 [1.51375898]]
0.10551144794587178


In [ ]:
import numpy as np

# simple program to calculate the chance of getting a move within a certain distance
samples = 300000000
test = np.random.uniform(-1,1, size = (samples,2))
test *= 3

index = np.all(test < 0.5, axis = -1)
index2 = np.all(test > -0.5,axis = -1)
print(test[index&index2].shape[0]/samples)

Tried basic chunking of the arrays, this still led to unstable results, there is no easy way to make this code run stably on my machine without carefully analysing the memory data of each array generation and so on and so forth. Not worth the time that I could spend looking at the actual physics.

Using a for loop would remove the memory issue but i would then be calculating upwards of 38 billion values with a for loop, which would take way too long.

Testing the random walk to find how many walkers to get a stable 95% value

In [26]:
disk_radius = 100
delta = 3
total_steps_lst = []
precision = 5
num_charges = 5
walkers = np.power(10,precision) # the number of walkers to generate to extract the data from
dims = 2 # the dimension of the random walk simulation
total_steps = 80000 # the initial total steps needed so that charges will move across the radius of the disk
steps = int(total_steps/num_charges) # get the step needed per charge
distances = walk(walkers, steps, dims, delta = delta) # return the random walk data
distances = np.linalg.norm(distances,axis=-1) # calculate the distances reached by each walker
reached = np.unique(np.where(distances > disk_radius)[0]) # find the index where each walker has reached the radius of the disk
print('random walk data', len(reached), len(reached)/walkers, int(total_steps)) # print the data

MemoryError: Unable to allocate 23.8 GiB for an array with shape (100000, 16000, 2) and data type float64

As with the calculation of the standard deviation stability to a number of decimal places, without rigourous rewriting of the program and splitting arrays carefully for everypossible charge number and for each system that the code is run on, as i use both my laptop and my computer, it isn't faesible to go up above 10^4 as my computer started crashing and I couldn't obtain any proper data.
I know the problem lies in generating very large arrays, and a mixture of memmapping the arrayss and feeding them through the harddisk can have a possible solution to working with this, I believe this is outside the scope of what a computational physics course should be doing, If I wanted to do this for some random project I would have chosen computer science.

Especially as these techniques aren't taught in the course!